In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score, f1_score
import itertools

In [2]:
file_path = r"C:\Users\jens.nilsen\OneDrive - Bouvet Norge AS\Documents\GitHub\trfkaipoc\data_2022-2025.csv"
df = pd.read_csv(file_path, sep=";")

In [3]:
df = df[df["EGS.VEDTAK.10670"].notna()]

In [4]:
df["Vedtak"]=df["EGS.VEDTAK.10670"]

In [5]:
df["EGS.VEDTAK.10670"].unique()

array(['Utvidet bruk', 'Midlertidig tillatelse', 'Varig tillatelse',
       'Endret bruk', 'Avslag'], dtype=object)

In [6]:
y = df["EGS.VEDTAK.10670"]
# Kod om tekstklassene til tall
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [7]:
features = [
    "ÅDT, total",
    "ÅDT, andel lange kjøretøy",
    "Fartsgrense",
    "Avkjørsel, holdningsklasse",
    "Funksjonsklasse",
    "Avkjørsler",
    "Trafikkulykker",
    "EGS.BRUKSOMRÅDE.1256", 
    "Kurvatur, horisontal", 
    "Kurvatur, stigning"
]

# Encode categorical features
df_encoded = pd.get_dummies(df[features])

In [26]:
y = y_encoded     # target
X = df_encoded # all other columns

In [27]:
X

,"ÅDT, total","ÅDT, andel lange kjøretøy",Fartsgrense,Avkjørsler,Trafikkulykker,"Avkjørsel, holdningsklasse_Lite streng","Avkjørsel, holdningsklasse_Meget streng","Avkjørsel, holdningsklasse_Mindre streng","Avkjørsel, holdningsklasse_Streng",Funksjonsklasse_B - Regionale hovedveger,...,EGS.BRUKSOMRÅDE.1256_Dyrkamark/jordbruk,EGS.BRUKSOMRÅDE.1256_Forretning,EGS.BRUKSOMRÅDE.1256_Fritidsaktivitet,EGS.BRUKSOMRÅDE.1256_Gardsbruk,EGS.BRUKSOMRÅDE.1256_Hytte,EGS.BRUKSOMRÅDE.1256_Industri,EGS.BRUKSOMRÅDE.1256_Naust,EGS.BRUKSOMRÅDE.1256_Service,EGS.BRUKSOMRÅDE.1256_Skog/skogbruk,EGS.BRUKSOMRÅDE.1256_Skole/Barnehage
0,550.0,11.0,80.0,11,1,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,293.0,7.0,60.0,8,1,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,1500.0,13.0,80.0,2,4,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,15000.0,6.0,50.0,9,50,False,False,False,True,True,...,False,True,False,False,False,False,False,False,False,False
4,2980.0,12.0,80.0,5,8,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,1300.0,6.0,30.0,21,0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
464,500.0,10.0,60.0,12,2,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
465,400.0,13.0,60.0,12,0,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
466,1600.0,15.0,80.0,8,6,False,False,False,True,True,...,False,False,False,False,False,False,False,False,True,False


In [11]:
len(y)

468

In [12]:
import pandas as pd
import numpy as np
import itertools

# Anta at df er DataFrame-en din
df_num = X.copy()

# Konverter boolske til 0/1
for col in df_num.select_dtypes(include=["bool"]).columns:
    df_num[col] = df_num[col].astype(int)

# Fjern kolonner uten variasjon
valid_cols = [c for c in df_num.columns if df_num[c].nunique() > 1]

# === 2-veis interaksjoner ===
inter_2_dict = {
    f"{f1}*{f2}": df_num[f1] * df_num[f2]
    for f1, f2 in itertools.combinations(valid_cols, 2)
}
interactions_2way = pd.concat(inter_2_dict, axis=1)

# === 3-veis interaksjoner ===
inter_3_dict = {
    f"{f1}*{f2}*{f3}": df_num[f1] * df_num[f2] * df_num[f3]
    for f1, f2, f3 in itertools.combinations(valid_cols, 3)
}
interactions_3way = pd.concat(inter_3_dict, axis=1)

inter_4_dict = {
    f"{f1}*{f2}*{f3}*{f4}": df_num[f1] * df_num[f2] * df_num[f3]* df_num[f4]
    for f1, f2, f3, f4 in itertools.combinations(valid_cols, 4)
}
interactions_4way = pd.concat(inter_4_dict, axis=1)

# Slå alt sammen
#df_with_interactions = pd.concat([df_num, interactions_2way, interactions_3way], axis=1)
#df_with_interactions = pd.concat([df_num, interactions_2way], axis=1)
df_with_interactions = pd.concat([df_num, interactions_2way, interactions_3way, interactions_4way], axis=1)

In [13]:
#df_with_interactions = pd.concat([df_num, interactions_2way, interactions_3way], axis=1)
print("Opprinnelige kolonner:", len(df_num.columns))
print("Totalt etter interaksjoner:", len(df_with_interactions.columns))

Opprinnelige kolonner: 26
Totalt etter interaksjoner: 17901


In [14]:
X=df_with_interactions
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [15]:
model = RandomForestClassifier(
    n_estimators=10000,
    class_weight='balanced_subsample',
    random_state=99,
    n_jobs=-1
)
model.fit(X, y)

# Get feature importances
importances = pd.Series(model.feature_importances_, index=X.columns)
importances_sorted = importances.sort_values(ascending=False)
importances
top_features = importances_sorted.index[:20]
X_top = X[top_features]

In [16]:
importances_sorted.index[:20]

Index(['ÅDT, total*Avkjørsler', 'ÅDT, total*ÅDT, andel lange kjøretøy',
       'ÅDT, total', 'ÅDT, total*ÅDT, andel lange kjøretøy*Fartsgrense',
       'ÅDT, total*ÅDT, andel lange kjøretøy*Avkjørsler',
       'ÅDT, total*Fartsgrense', 'ÅDT, total*Kurvatur, stigning',
       'Fartsgrense*Avkjørsler',
       'ÅDT, total*ÅDT, andel lange kjøretøy*Fartsgrense*Avkjørsler',
       'Avkjørsler*Kurvatur, horisontal',
       'Fartsgrense*Avkjørsler*Kurvatur, horisontal',
       'ÅDT, andel lange kjøretøy*Avkjørsler*Kurvatur, horisontal',
       'ÅDT, total*Fartsgrense*Avkjørsler',
       'ÅDT, andel lange kjøretøy*Fartsgrense*Avkjørsler',
       'ÅDT, total*ÅDT, andel lange kjøretøy*Kurvatur, stigning',
       'ÅDT, total*ÅDT, andel lange kjøretøy*Fartsgrense*Kurvatur, stigning',
       'ÅDT, andel lange kjøretøy*Fartsgrense*Avkjørsler*Kurvatur, horisontal',
       'Kurvatur, stigning', 'ÅDT, total*Kurvatur, horisontal',
       'ÅDT, total*Fartsgrense*Kurvatur, stigning'],
      dtype='object'

In [17]:
X=X_top

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [19]:
model = RandomForestClassifier(n_estimators=10000,n_jobs=-1,class_weight='balanced_subsample')
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced_subsample', n_estimators=10000,
                       n_jobs=-1)

In [20]:
model = CatBoostClassifier(
    iterations=10000,         # antall trær
    learning_rate=0.01,
    depth=6,
    loss_function='MultiClass',
    random_seed=42,
    verbose=100
)

In [21]:
model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=500)

0:	learn: 1.6057200	test: 1.6067003	best: 1.6067003 (0)	total: 156ms	remaining: 25m 59s
100:	learn: 1.3292731	test: 1.4126072	best: 1.4126072 (100)	total: 888ms	remaining: 1m 27s
200:	learn: 1.1886538	test: 1.3425621	best: 1.3425621 (200)	total: 1.6s	remaining: 1m 18s
300:	learn: 1.0950087	test: 1.3115375	best: 1.3115375 (300)	total: 2.34s	remaining: 1m 15s
400:	learn: 1.0223699	test: 1.2933644	best: 1.2933644 (400)	total: 3.12s	remaining: 1m 14s
500:	learn: 0.9573118	test: 1.2830187	best: 1.2828512 (496)	total: 3.94s	remaining: 1m 14s
600:	learn: 0.8990331	test: 1.2778965	best: 1.2776462 (588)	total: 4.71s	remaining: 1m 13s
700:	learn: 0.8459014	test: 1.2753404	best: 1.2748900 (682)	total: 5.53s	remaining: 1m 13s
800:	learn: 0.7962017	test: 1.2727981	best: 1.2727834 (799)	total: 6.27s	remaining: 1m 11s
900:	learn: 0.7492374	test: 1.2721312	best: 1.2714278 (849)	total: 7s	remaining: 1m 10s
1000:	learn: 0.7043095	test: 1.2723693	best: 1.2714278 (849)	total: 7.72s	remaining: 1m 9s
1100:	

In [22]:
yps=model.predict_proba(X_test)

In [23]:
y_s=model.predict(X_test)

In [24]:
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred, target_names=le.classes_))

# Konfusjonsmatrise
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix – Multiclass Classification")
plt.show()

                        precision    recall  f1-score   support

                Avslag       0.00      0.00      0.00         8
           Endret bruk       0.00      0.00      0.00        14
Midlertidig tillatelse       0.71      0.30      0.43        33
          Utvidet bruk       0.39      0.67      0.49        70
      Varig tillatelse       0.35      0.30      0.32        63

              accuracy                           0.40       188
             macro avg       0.29      0.26      0.25       188
          weighted avg       0.39      0.40      0.37       188



C:\Users\jens.nilsen\python\WPy64-31230\python-3.12.3.amd64\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\jens.nilsen\python\WPy64-31230\python-3.12.3.amd64\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\jens.nilsen\python\WPy64-31230\python-3.12.3.amd64\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

NameError: name 'confusion_matrix' is not defined

In [ ]:
importances = pd.Series(model.feature_importances_, index=X_train.columns)
importances_sorted = importances.sort_values(ascending=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Top 5 features
top_features = importances_sorted.index[:3]

# Lag subset med target
X_top = X[top_features].copy()
X_top['target'] = le.inverse_transform(y_encoded)  # tekstklassene eller tallkodene

# Definer en palett med 5 farger (kan justeres)
palette = sns.color_palette("Set1", n_colors=5)  # Set1 gir tydelige farger

# Pairwise scatter plots
sns.pairplot(
    X_top,
    vars=top_features,
    hue='target',            # farge etter target
    palette=palette,
    plot_kws={'alpha': 0.7}
)
plt.suptitle("Pairwise Plots of Top Features", y=1.02)
plt.show()
